In [27]:
from joblib import dump, load
import pandas as pd
import numpy as np

In [28]:
feature_intervals = {
    'aspect': np.arange(0, 370, 10),
    'slope' :  np.arange(0, 70, 2),
    #'forest-height': np.arange(0, 60, 1),
    'elevation': np.arange(0, 650, 10),
    'tree_cover': np.arange(20, 105, 2),
    'curvature': np.arange(-20, 20, 0.5),
    # 'curvature-plan': np.arange(-13, 13, 0.5),
    # 'curvature-prof': np.arange(-13, 13, 0.5),
    'patches': np.array([0, 10, 100, 1000, 10000, 100000, 1000000, 1500000, 2000000]),
    #'patches': np.array([0] + [np.exp(j) for j in range(1, 16)]),
    'morphology': np.arange(0, 12, 1)
}


In [29]:
features = load('output.dat')

In [30]:
features=pd.DataFrame(features)

In [31]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   aspect      992676 non-null   float64
 1   curvature   992676 non-null   float64
 2   elevation   992676 non-null   float64
 3   morphology  992676 non-null   float64
 4   slope       992676 non-null   float64
 5   tree_cover  992546 non-null   float64
 6   patches     1000000 non-null  float32
 7   wfall       1000000 non-null  bool   
dtypes: bool(1), float32(1), float64(6)
memory usage: 50.5 MB


In [32]:
features.dropna(axis=0, inplace=True)

In [16]:
features = features[features.elevation!=32768]

In [17]:
features.shape

(155143, 8)

In [18]:
features

,aspect,curvature,elevation,morphology,slope,tree_cover,patches,wfall
23,225.0000,-inf,84.0,8.0,12.601562,97.0,0.0,False
24,357.2500,inf,427.0,2.0,13.906250,86.0,0.0,False
29,294.2500,inf,117.0,4.0,7.339844,85.0,0.0,False
30,115.3750,-inf,85.0,6.0,4.871094,86.0,0.0,False
36,221.3750,inf,266.0,5.0,20.187500,62.0,0.0,False
...,...,...,...,...,...,...,...,...
992530,10.7500,inf,340.0,5.0,18.062500,0.0,0.0,False
992532,281.7500,-inf,391.0,5.0,26.234375,0.0,0.0,False
992534,105.9375,inf,145.0,5.0,6.734375,0.0,0.0,False
992538,142.2500,-inf,682.0,5.0,24.531250,0.0,0.0,False


In [33]:
features[features.wfall]

,aspect,curvature,elevation,morphology,slope,tree_cover,patches,wfall
152,138.62500,-inf,92.0,3.0,5.250000,87.0,7330.468750,True
335,37.40625,inf,76.0,3.0,4.957031,85.0,4134.810059,True
562,-inf,-inf,32768.0,7.0,-inf,0.0,7781.903809,True
609,-1.00000,-0.0,129.0,7.0,0.000000,0.0,3385.572998,True
830,-inf,-inf,32768.0,7.0,-inf,0.0,2906.431396,True
...,...,...,...,...,...,...,...,...
991082,-inf,-inf,32768.0,7.0,-inf,85.0,404527.531250,True
991770,-inf,-inf,32768.0,7.0,-inf,0.0,427.199036,True
992008,-inf,-inf,32768.0,7.0,-inf,0.0,1472.554443,True
992077,335.50000,inf,502.0,4.0,12.085938,0.0,98864.632812,True
